In [6]:
__version__ = '2.0'
__author__  = "Robert Matern (r.matern@stud.uni-hannover.de)"
__date__    = ''
__url__     = ''
__copyright__ = "(C) 2015 Robert Matern"

In [7]:
import numpy as np
import pandas as pd
import matplotlib as plt

#Dynamisches Programm

## Algorithmus

In [8]:
def DP_Time(solutions, conditions, products, resources, capacities, consumtions, times, revenues, probs, stocks, max_stocks, performance_times):
    '''Berechnung der Erwartungswerte des Auftragsannahmeproblems inkl. Lagerhaltung.'''
    
    # Regeneration der Kapazitäten in dieser Rechnung nicht möglich!!!
    
    # Leere Wert für den Erwartungswert des aktuellen Systemzustands.
    value = 0
    
    #Aktueller Systemzustand wird generiert.
    condition = np.hstack((capacities[1:],stocks[1:],times[0]))
    
    #Aktueller Systemzustand wird in dem 'np.array' aller möglichen Systemzustände gesucht.
    state = np.where((conditions == condition).all(axis=1))[0][0]
    
    # Memofunktion: Die Rechnung wird nur fortgeführt, sofern es nicht schon berechnet wurde.
    if state not in solutions:
                
        # Sofern es sich nicht um einen Endknoten des Entscheidungsbaums handelt,
        # werden folgende Schritte eingeleitet:
        if times[0]!=0:
            # Der Erwartungswert t-1 ohne Akeptanz wird ermittelt und im Wert 'reject' gespeichert.
            reject = DP_Time(solutions, conditions, products, resources, capacities, consumtions, times[1:], revenues, probs, stocks, max_stocks, performance_times)
            
            # Für den Erwartungswert t-1 mit Akzeptanz werden Numpy-Arrays in der Länge der Anzahl an Produkten erstellt.
            accept = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            oc = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            max_order = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            
            # Da hier auch ein Lagerhaltungbestand berücksichtigt ist, wird auch hierführ Numpy-Arrays erstellt. 
            accept_stock = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            oc_stock = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            max_order_stock = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            
            # Da hier auch ein Lagerhaltungbestand berücksichtigt ist, wird auch hierführ Numpy-Arrays erstellt. 
            accept_workup = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            oc_workup = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            max_order_workup = np.zeros(shape=(len(products[1:])), dtype=np.float16)
            
            # For-Schleife über alle Produkte, sofern die Kapazitäten keinen negativen Werte annehmen.
            product = (j for j in products[1:] if probs[j][times[0]] > 0)
            for j in product:
                
                # Ermittlung der Leistungsperiode
                all_times = np.hstack(performance_times)
                lp = np.divide(np.where((all_times == times[0]))[0][0], len(performance_times[0]))+1
                    
                change = capacities-consumtions[j]
                decrease_stock = stocks-consumtions[j]
                
                if np.all((change) >= np.float32(0)):
                    # Initialisierung der Rechnung für t-1 mit Akeptanz jeweils für ein Produkt j.
                    accept_j = DP_Time(solutions, conditions, products, resources, change, consumtions, times[1:], revenues, probs, stocks, max_stocks, performance_times)
                    oc[j-1] = reject-accept_j
                    max_order[j-1] = max(revenues[j]-reject+accept_j, 0)
                    accept[j-1] = probs[j][times[0]]*max_order[j-1]            

                else:    
                    # Erwartungswert für ein Produkt j enspricht
                    # der Grenzbedingung V(c,st)=-∞, falls n[j] < 0.
                    accept[j-1] = np.float32(0)

                if np.all((decrease_stock) >= np.float32(0)):
                    # Initialisierung der Rechnung für t-1 mit Akeptanz jeweils für ein Produkt j.
                    accept_stock_j = DP_Time(solutions, conditions, products, resources, capacities, consumtions, times[1:], revenues, probs, decrease_stock, max_stocks, performance_times)
                    oc_stock[j-1] = reject-accept_stock_j
                    max_order_stock[j-1] = max(revenues[j]-reject+accept_stock_j, 0)
                    accept_stock[j-1] = probs[j][times[0]]*max_order_stock[j-1]
                else:    
                    # Erwartungswert für ein Produkt j enspricht
                    # der Grenzbedingung V(c,st)=-∞, falls n[j] < 0.
                    accept_stock[j-1] = np.float32(0)

                increase_stock = np.copy(stocks)
                increase_stock[lp:] = increase_stock[lp:]+np.sum(consumtions[j])
                if np.all((change) >= np.float32(0)) and np.all((increase_stock) <= (max_stocks)):
                    # Initialisierung der Rechnung für t-1 mit Akeptanz jeweils für ein Produkt j.
                    accept_workup_j = DP_Time(solutions, conditions, products, resources, change, consumtions, times[1:], revenues, probs, increase_stock, max_stocks, performance_times)
                    oc_workup[j-1] = reject-accept_workup_j
                    max_order_workup[j-1] = max(accept_workup_j-reject, 0)
                    accept_workup[j-1] = probs[j][times[0]]*max_order_workup[j-1]
                else:    
                    # Erwartungswert für ein Produkt j enspricht
                    # der Grenzbedingung V(c,st)=-∞, falls n[j] < 0.
                    accept_workup[j-1] = np.float32(0)
            
            # Summierung der Erwartungswerte ohne und mit Akzeptanz.
            value = np.float32(reject + np.sum(accept) + np.sum(accept_stock) + np.sum(accept_workup))
            # Einzelne Optionen werden nach ihrem erw. Ertrag indiziert.
            opt_order = max(max_order)
            opt_order_stock = max(max_order_stock)
            opt_order_workup = max(max_order_workup)
            opt = [opt_order, opt_order_stock, opt_order_workup]
            index = opt.index(max(opt))
            if index == 0 and np.all(opt == np.float32(0)):
                solutions[state] = [conditions[state], np.float32(0), 0, 'KA', 0, [max_order!=0, max_order_stock!=0, max_order_workup!=0]] 
            elif index ==0:
                order = np.where((max_order == opt_order))[0][0]
                solutions[state] = [conditions[state], value, order+1, 'AA', revenues[order+1]-oc[order], [max_order!=0, max_order_stock!=0, max_order_workup!=0]]
            elif index ==1:
                order_stock = np.where((max_order_stock == opt_order_stock))[0][0]
                solutions[state] = [conditions[state], value, order_stock+1, 'LE', revenues[order_stock+1]-oc_stock[order_stock], [max_order!=0, max_order_stock!=0, max_order_workup!=0]]
            elif index==2:
                order_workup = np.where((max_order_workup == opt_order_workup))[0][0]
                solutions[state] = [conditions[state], value, order_workup+1, 'IH', revenues[order_workup+1]-oc_workup[order_workup], [max_order!=0, max_order_stock!=0, max_order_workup!=0]]
            else:
                solutions[state] = [conditions[state], np.float32(0), 0, 'KA', 0, [max_order!=0, max_order_stock!=0, max_order_workup!=0]]
            print 'Bestände:', solutions[state][0][:-1], '- Periode:', solutions[state][0][-1], '- Erwartungswert:', solutions[state][1], '- Optimale Politik (AA):', solutions[state][5][0], '- Optimale Politik (LE):', solutions[state][5][1], '- Optimale Politik (IH):', solutions[state][5][2]
        # Sofern es sich um einen Endknoten des Entscheidungsbaums handelt, werden folgende Schritte eingeleitet:
        else:
            # Erwartungswert enspricht der Grenzbedingung V(c,0)=0, für n >= 0.
            value = np.float32(0)
            # Ein Endzustand wird mit einem Erwartungswert 0 in das Dict "solutions" gespeichert.
            solutions[state] = [conditions[state], value, 0, 'KA', 0, [[False]*len(products[1:]), [False]*len(products[1:]), [False]*len(products[1:])]]
            print 'Bestände:', solutions[state][0][:-1], '- Periode:', solutions[state][0][-1], '- Erwartungswert:', solutions[state][1], '- Optimale Politik (AA):', solutions[state][5][0], '- Optimale Politik (LE):', solutions[state][5][1], '- Optimale Politik (IH):', solutions[state][5][2]        
            
    # Memofunktion: Sofern das Ergebnis breits berechnet wurde, wird der Wert aus dem Dict "solutions" verwendet.
    else:
        value = solutions[state][1]
               
    return value

## Erstellung der Struktur als NetworkX-Graph

In [9]:
import networkx as nx

def Structure_Transform(solutions, conditions, products, resources, consumtions, revenues, probs, stocks, max_stocks, performance_times):
    '''Generierung der Stuktur der Problemstellung.'''
    
    # MultiDiGraph erstellen.
    graph=nx.MultiDiGraph()
    
    # Knoten für alle Lösungen (solutions) erstellen.
    for key in solutions.iterkeys():
        graph.add_node(key, label=solutions[key][0], value=solutions[key][1],
                   capacity=solutions[key][0][:(len(resources)-1)], time=solutions[key][0][-1],
                      stock=solutions[key][0][(len(resources)-1):-1])
    
    # Endknoten zur Nutzung der NetworkX-Algorithmen anlegen.
    graph.add_node("end", label='end', op=0, value=0,
                   capacity=0, stock=0, time=0)
    
    # Kanten erstellen.
    # Schleife über alle Lösungen.
    for i in solutions.iterkeys():
        # Handelt es sich um einen Zeitpunkt 0, dann wird der Systemzustand mit dem 'end'-Knoten verbunden.
        if solutions[i][0][-1] == 0:
            graph.add_edge(i, "end", key=0, modus=0, weight=0, revenue=0, time=0)
        # Sonst führe die Schleife aus.
        else:
            # Aufgrund der differenzierten Auftragsannahme erfolgt eine Schleife über alle Produkte.
            for j in products:
                # Kapazitätsveränderung aufgrund der Anfragen nach Produkt 'j' wird erfasst.
                all_times = np.hstack(performance_times)
                lp = np.divide(np.where((all_times == solutions[i][0][-1]))[0][0], len(performance_times[0]))  
                change = np.copy(solutions[i][0][:len(resources)-1])
                change = change-consumtions[j][1:]
                # Keine negativen Kapazitätsbestände möglich.
                if np.all((change) >= np.float32(0)): 
                        # Zielzustand wird ermittelt. 
                        reduction = np.hstack((change,solutions[i][0][len(resources)-1:-1],solutions[i][0][-1]-1))
                        state = np.where((conditions == reduction).all(axis=1))[0][0]
                        # Zielzustand zur Ermittlung der Opportunitätskosten wird ermittelt.
                        no_reduction = np.append(solutions[i][0][:-1], solutions[i][0][-1]-1)
                        state2 = np.where((conditions == no_reduction).all(axis=1))[0][0]
                        # Prüfung und Verknüpfung der Anfragen     
                        if j == 0:
                            graph.add_edge(i, state, key=j, modus='KA', label='KA', weight=revenues[j]-solutions[state2][1]+solutions[state][1], weight_goal=solutions[state][1], revenue=revenues[j], goal=solutions[state][0], time=solutions[i][0][-1])
                        elif probs[j][solutions[i][0][-1]] > 0:
                            # Abfrage, ob der Revenue höher ist als die Opportunitätskosten.
                            if revenues[j] >= solutions[state2][1]-solutions[state][1]:
                                graph.add_edge(i, state, key=j, modus='AA', op=1, label=str(j)+'-AA', weight=revenues[j]-solutions[state2][1]+solutions[state][1], weight_goal=solutions[state][1], revenue=revenues[j], goal=solutions[state][0], time=solutions[i][0][-1])
                            else:
                                graph.add_edge(i, state, key=j, modus='AA', op=0, label=str(j)+'-AA', style="dotted", weight=0, weight_goal=solutions[state][1], revenue=0, goal=solutions[state][0], time=solutions[i][0][-1])

                # Lagerveränderung aufgrund der Anfragen nach Produkt 'j' wird erfasst.
                decrease_stock = np.copy(solutions[i][0][(len(resources)-1):-1])
                decrease_stock = decrease_stock-consumtions[j][1:]
                if np.all((decrease_stock) >= np.float32(0)): 
                        # Zielzustand wird ermittelt. 
                        reduction_stock = np.hstack((solutions[i][0][:len(resources)-1],decrease_stock,solutions[i][0][-1]-1))
                        state_stock = np.where((conditions == reduction_stock).all(axis=1))[0][0]
                        # Zielzustand zur Ermittlung der Opportunitätskosten wird ermittelt.
                        no_reduction_stock = np.append(solutions[i][0][:-1], solutions[i][0][-1]-1)
                        state2_stock = np.where((conditions == no_reduction_stock).all(axis=1))[0][0]
                        # Prüfung und Verknüpfung der Anfragen     
                        if j == 0:
                            graph.add_edge(i, state_stock, key=j, modus='KA', label='KA', weight=revenues[j]-solutions[state2_stock][1]+solutions[state_stock][1], weight_goal=solutions[state_stock][1], revenue=revenues[j], goal=solutions[state_stock][0], time=solutions[i][0][-1])
                        elif probs[j][solutions[i][0][-1]] > 0:
                            # Abfrage, ob der Revenue höher ist als die Opportunitätskosten.
                            if revenues[j] >= solutions[state2_stock][1]-solutions[state_stock][1]:
                                graph.add_edge(i, state_stock, key=j, modus='LE', op=1, label=str(j)+'-LE', weight=revenues[j]-solutions[state2_stock][1]+solutions[state_stock][1], weight_goal=solutions[state_stock][1], revenue=revenues[j], goal=solutions[state_stock][0], time=solutions[i][0][-1])
                            else:
                                graph.add_edge(i, state_stock, key=j, modus='LE', op=0, label=str(j)+'-LE', style="dotted", weight=0, weight_goal=solutions[state_stock][1], revenue=0, goal=solutions[state_stock][0], time=solutions[i][0][-1])
                    
                # Lagererhöhung aufgrund der Anfragen nach Produkt 'j' wird erfasst.
                increase_stock = np.copy(solutions[i][0][(len(resources)-1):-1])
                increase_stock[lp:] = increase_stock[lp:]+np.sum(consumtions[j][1:])
                if np.all((change) >= np.float32(0)) and np.all((increase_stock) <= (max_stocks[1:])): 
                    # Zielzustand wird ermittelt. 
                    propagation_stock = np.hstack((change,increase_stock,solutions[i][0][-1]-1))
                    state_storage = np.where((conditions == propagation_stock).all(axis=1))[0][0]
                    # Zielzustand zur Ermittlung der Opportunitätskosten wird ermittelt.
                    no_reduction_storage = np.append(solutions[i][0][:-1], solutions[i][0][-1]-1)
                    state2_storage = np.where((conditions == no_reduction_storage).all(axis=1))[0][0]
                    # Prüfung und Verknüpfung der Anfragen
                    if j == 0:
                        graph.add_edge(i, state_storage, key=j, modus='KA', label='KA', weight=revenues[j]-solutions[state2_storage][1]+solutions[state_storage][1], weight_goal=solutions[state_storage][1], revenue=revenues[j], goal=solutions[state_storage][0], time=solutions[i][0][-1])
                    elif probs[j][solutions[i][0][-1]] > 0:
                        # Abfrage, ob der Revenue höher ist als die Opportunitätskosten.
                        if revenues[j] >= solutions[state2_storage][1]-solutions[state_storage][1]:
                            graph.add_edge(i, state_storage, key=j, modus='IH', op=1, label=str(j)+'-IH', weight=solutions[state_storage][1]-solutions[state2_storage][1], weight_goal=solutions[state_storage][1], revenue=0, goal=solutions[state_storage][0], time=solutions[i][0][-1])
                        else:
                            graph.add_edge(i, state_storage, key=j, modus='IH', op=0, label=str(j)+'-IH', style="dotted", weight=0, weight_goal=solutions[state_storage][1], revenue=0, goal=solutions[state_storage][0], time=solutions[i][0][-1])

    return graph

## Ermittlung der optimalen Politik

In [10]:
def Best_Politic_Stock(graph, times, resources, stocks, solutions):
    '''Ermittlung der optimalen Politik ohne Berücksichtigung der Nachfrage.'''
    
    print 'Optimalen Politik zum Zeitpunkt "t" und unter Beachtung der Restkapazitäten "c[h]":', '\n'
        
    # List mit topologischer Sortierung des Graphen wird erstellt.
    list = nx.topological_sort(graph)
    
    # Leere Listen für die 'Pandas'-Tabellenspalten erstellen
    db_sol = []
    db_time = []
    db_res = []
    db_sto = []
    db_suc = []
    db_bp = []
    db_mo = []
    db_oc = []
    #db_sucp = []
     
    # Schleife über alle Knoten der topologisch sortierten Liste.
    for node in list:
        # Listen für 'Pandas'-Tabellenspalten erstellen.
        db_sol.append(graph.node[node]['value'])
        db_time.append(graph.node[node]['time'])
        db_suc.append(graph.successors(node))
        # Ist die Schleife zum Endknoten der Liste gekommen, dann wird sie unterbrochen.
        if node == 'end':
            db_bp.append(0)
            db_mo.append('-')
            db_oc.append(0)
            #db_sucp.append(0)
            break
        # Ist die Schleife zu einem Endzeitpunkt gekommen, dann wird sie übersprungen.
        elif graph.node[node]['time'] == 0:
            db_bp.append(0)
            db_mo.append('-')
            db_oc.append(0)
            #db_sucp.append(0)
            continue
        # Sonst werden die möglichen Auftragsannahmen des Knotens in eine Liste geschrieben.
        else:
            db_bp.append(solutions[node][2])
            if len(solutions[node])==5:
                db_mo.append(solutions[node][3])
            else:
                db_mo.append(str(solutions[node][3]))
            db_oc.append(solutions[node][4])
            #db_sucp.append(graph.successors(node)[solutions[node][3]-1])
        
        for suc in graph.successors(node):
            for order in graph.edge[node][suc].iterkeys():
                # Boolescher Wert 'not_best_politic' zur Ermittlung des Pfads wird angelegt.
                if len(solutions[node])==5:
                    if order == solutions[node][2] and graph.edge[node][suc][order]['modus'] == solutions[node][3]:
                        graph.edge[node][suc][order]['not_best_politic']=False
                        graph.edge[node][suc][order]['style']='bold'                   
                    else:
                        graph.edge[node][suc][order]['not_best_politic']=True
                else:
                    if order == solutions[node][2] and str(graph.edge[node][suc][order]['modus']) == str(solutions[node][3]):
                        graph.edge[node][suc][order]['not_best_politic']=False
                        graph.edge[node][suc][order]['style']='bold' 
                    else:
                        graph.edge[node][suc][order]['not_best_politic']=True
            
    
    # Spalten für die Ressourcenkapazitäten erstellen.
    for res in resources[1:]:
        db_res.append('$c^{'+str(res)+'}$')
    db_cap = []
    for c in range(len(db_res)):
        cap = []
        for n in list:
            if n == 'end':
                cap.append(0)
            else:
                cap.append(graph.node[n]['capacity'][c])
        db_cap.append(cap)

    # Spalten für die Lagerbestände erstellen.
    for sto in range(len(stocks[1:])):
        db_sto.append('$y^{'+str(sto+1)+'}$')
    db_stos = []
    for s in range(len(db_sto)):
        stos = []
        for n in list:
            if n == 'end':
                stos.append(0)
            else:
                stos.append(graph.node[n]['stock'][s])
        db_stos.append(stos)

    # Tabelle erstellen                
    db = pd.DataFrame(index=list)
    for i, res in enumerate(db_res):
        db[res] = db_cap[i]
    for k, sto in enumerate(db_sto):
        db[sto] = db_stos[k]
    db['$t$'] = db_time
    db['ExpValue'] = db_sol
    db['Successor'] = db_suc
    db['$j^*$'] = db_bp
    db['$m$'] = db_mo
    #db['Best Successor'] = db_sucp
    db['$r_{j}-OC_{j}$'] = db_oc
    
    
    # Tabelle ausgeben.
    db = db[:-1]
    db.to_csv('Table_Graph.csv')
    print db, '\n'
    
    
    # Optimaler Pfad wird mittels Dijkstra-Algorithmus geschrieben.
    path = nx.dijkstra_path(graph, source=list[0], target=list[-1], weight='not_best_politic')[:-1]
    
    # Tabelle erstellen.
    rev = [0] * len(times)
    dec = [0] * len(times)
    mod = [0] * len(times)
    fr = [0] * len(times)
    to = [0] * len(times)
    for i, node in enumerate(path[:-1]):
        for order in graph.edge[node][path[i+1]].iterkeys():
            if graph.edge[node][path[i+1]][order]['not_best_politic'] == False:
                rev[graph.node[node]['time']] = graph.edge[node][path[i+1]][order]['revenue']
                dec[graph.node[node]['time']] = order
                mod[graph.node[node]['time']] = graph.edge[node][path[i+1]][order]['modus']
                fr[graph.node[node]['time']] = path[i]
                to[graph.node[node]['time']] = path[i+1]

    df = pd.DataFrame(dec[::-1], index=times, columns = ['Opt. Politic'])
    df['Modus'] = mod[::-1]
    df['from'] = fr[::-1]
    df['to'] = to[::-1]
    df['Revenue'] = rev[::-1]
    df['Cum. Rev.'] = df.Revenue.cumsum()
    df = df[:-1]
    df.to_csv('Table_Politic.csv')
    print df, '\n'
        
    return

In [11]:
def Opt_Politic_Stock(solutions, resources, stocks, products):
    
    db = pd.DataFrame(index=solutions)
    
    db_res = []
    db_sto = []
    db_time = []
    db_sol = []
    db_aa = []
    db_le = []
    db_ih = []
    
    
    # Spalten für die Ressourcenkapazitäten erstellen.
    for res in resources[1:]:
        db_res.append('$c^{'+str(res)+'}$')
    db_cap = []
    for c in range(len(db_res)):
        cap = []
        for s in solutions:
            cap.append(solutions[s][0][c])
        db_cap.append(cap)

    # Spalten für die Lagerbestände erstellen.
    for sto in range(len(stocks[1:])):
        db_sto.append('$y^{'+str(sto+1)+'}$')
    db_stos = []
    for i in range(len(db_sto)):
        stos = []
        for s in solutions:
            stos.append(solutions[s][0][i+len(resources[1:])])
        db_stos.append(stos)
    
    for s in solutions:
        db_time.append(solutions[s][0][-1])
        db_sol.append(solutions[s][1])

    # Spalten für die Optionen.
    for j in range(len(products[1:])):
        db_aa.append('$j^{'+str(j+1)+'}_{AA}$')
        db_le.append('$j^{'+str(j+1)+'}_{LE}$')
        db_ih.append('$j^{'+str(j+1)+'}_{IH}$')              
    db_aas = []
    db_les = []
    db_ihs = []
    for i in range(len(products[1:])):
        aas = []
        les = []
        ihs = []
        for s in solutions:
            aas.append(solutions[s][5][0][i]*1)
            les.append(solutions[s][5][1][i]*1)
            ihs.append(solutions[s][5][2][i]*1)
        db_aas.append(aas)
        db_les.append(les)
        db_ihs.append(ihs)
     
    for i, res in enumerate(db_res):
        db[res] = db_cap[i]
    for k, sto in enumerate(db_sto):
        db[sto] = db_stos[k]
    db['$t$'] = db_time
    db['ExpValue'] = db_sol
    for i, aa in enumerate(db_aa):
        db[aa] = db_aas[i]
    for i, le in enumerate(db_le):
        db[le] = db_les[i]
    for i, ih in enumerate(db_ih):
        db[ih] = db_ihs[i]
    
    # Tabelle ausgeben.
    db = db[:-1]
    db.to_csv('Table_Optimal.csv')
    
    return db
    